Import required libraries

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install langchain openai
!pip install langchain-openai
!pip install --upgrade langchain langchain-community

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nq4p9g5m/unsloth_b43ceaf518e44d0598dd7c1b68ee5468
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nq4p9g5m/unsloth_b43ceaf518e44d0598dd7c1b68ee5468
  Resolved https://github.com/unslothai/unsloth.git to commit 18e1bb545784aa6075ad356d62e49f05cef084ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 11.6 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.16
    Uninstalling langchain-0.3.16:
      Successfully uninstalled langchain-0.3.16


In [2]:
# Login
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Agent libs
from langchain.agents import initialize_agent, Tool
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from pydantic import BaseModel
# Environment libs
import os

from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TRANSLATOR_MODEL = "RodrigoSalazar-U/T-Translator-llama-mix-D3-e1"
#assert OPENAI_API_KEY, "Please set the OPENAI_API_KEY environment variable"
#assert TRANSLATOR_MODEL, "Please set the TRANSLATOR_MODEL environment variable"

Load local model

In [5]:
"""
from unsloth import FastLanguageModel

print(f"Using model: {TRANSLATOR_MODEL}")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = TRANSLATOR_MODEL,
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

FastLanguageModel.for_inference(model)
"""

'\nfrom unsloth import FastLanguageModel\n\nprint(f"Using model: {TRANSLATOR_MODEL}")\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    model_name = TRANSLATOR_MODEL,\n    max_seq_length = 4096,\n    dtype = None,\n    load_in_4bit = True\n)\n\nFastLanguageModel.for_inference(model)\n'

## Agents

- Generator:
  - Context: Seed sentences
  - Action: Generate a combination of sentences
- Translator:
  - Context: Example translations
  - Action: Translate source sentence to target language

In [ ]:

"""
END_TAG = "[/"
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=1024,
                min_new_tokens=3, # Force generation
                #stop_strings=END_TAG,  # Use list for stop strings to handle multiple inputs
                use_cache=True,
                do_sample=False # Deterministic
)
hf = HuggingFacePipeline(pipeline=pipe)
"""

hf = HuggingFacePipeline.from_model_id(
    model_id=TRANSLATOR_MODEL,
    task="text-generation",
    device=0,
    pipeline_kwargs={
        "max_new_tokens": 1024,
        "min_new_tokens": 3,
        "use_cache" : True,
        "do_sample" : False,
    },
)


In [4]:
generator = ChatOpenAI(model="gpt-4o-mini")

def generate_text(seed_sentences: list[str]) -> str:
    prompt = PromptTemplate.from_template("Sentence {i}: {sentence}")
    examples = [{"i": i, "sentence": sentence} for i, sentence in enumerate(seed_sentences)]
    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=prompt,
        prefix="Create a new different sentence that contains words from the following sentences. The new sentence should make logical sense and be grammatically correct sentence. The new sentence should be the same length as the original sentences.\n",
        suffix="\nNew sentence:"
    )
    prompt_text = few_shot_prompt.format()
    # DEBUG: print(prompt_text)
    response = generator.invoke(prompt_text)
    return response.content # type: ignore

translator = hf

class TranslationInput(BaseModel):
    text: str
    examples: list[tuple[str, str]]

def translate_text(input_data : TranslationInput) -> str:
    translation_prompt = PromptTemplate.from_template("[EN]{source_text}[/EN]\n[ANG]{target_text}[/ANG]")
    translation_examples = [{"source_text": source_text, "target_text": target_text} for source_text, target_text in input_data.examples]

    few_shot_prompt = FewShotPromptTemplate(
        examples=translation_examples,
        example_prompt=translation_prompt,
        prefix="Translate the following text from English to Anglo-Saxon:\n",
        suffix=f"\n[EN]{input_data.text}[/EN]\n[ANG]",
        input_variables=["text"]
    )
    prompt_text = few_shot_prompt.format(text=input_data.text)
    # DEBUG: print(prompt_text)
    response = translator.invoke(prompt_text)
    return response # type: ignore

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Tools

generator_tool = Tool(
    name="generate_text",
    description="Generate a new sentence that contains words from the given sentences.",
    input_variables=["seed_sentences"],
    output_variables=["new_sentence"],
    func=generate_text
)

translator_tool = Tool(
    name="translate_text",
    description="Translate the given text from English to Anglo-Saxon.",
    input_variables=["data"],
    output_variables=["translation"],
    func=translate_text
)


In [6]:
# Initalize pipeline/agents
def agent_pipeline(example_sentences: list[tuple[str, str]]):

    seed_sentences = [sentence[0] for sentence in example_sentences]

    new_english_sentence = generator_tool.run({"seed_sentences":seed_sentences})

    data = TranslationInput(text=new_english_sentence, examples=example_sentences)

    new_anglo_saxon_sentence = translator_tool.run({"data":data})
    return new_english_sentence, new_anglo_saxon_sentence


In [9]:
# USAGE
examples=[
  ("now was one of the twelve of christ's companions, who was called judas, seduced by the instigation of the devil, and he went to the jewish people, and consulted with them how he might betray christ unto them.", "ða wearð an ðæra twelfa cristes geferena, se wæs iudas gehaten, ðurh deofles tihtinge beswicen, and he eode to ðam iudeiscum folce, and smeade wið hi, hu he crist him belæwan mihte."),
  ("in that battle fell the power and dignity of the medes.", "and on ðæm gefeohte meða cræft and heora duguð gefeoll."),
  ("46 shillings shall be given as compensation for a wound on the head, if both bones are pierced.", "xl. heafodwunde to bote, gif ða ban beoð butu ðyrel, xxx scillinga geselle him mon.")
]

new_english_sentence, new_anglo_saxon_sentence = agent_pipeline(examples)
print(new_english_sentence)
print(new_anglo_saxon_sentence)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In that moment, one of the twelve companions consulted with the medes, who were called to betray dignity for the power of compensation.
Translate the following text from English to Anglo-Saxon:


[EN]now was one of the twelve of christ's companions, who was called judas, seduced by the instigation of the devil, and he went to the jewish people, and consulted with them how he might betray christ unto them.[/EN]
[ANG]ða wearð an ðæra twelfa cristes geferena, se wæs iudas gehaten, ðurh deofles tihtinge beswicen, and he eode to ðam iudeiscum folce, and smeade wið hi, hu he crist him belæwan mihte.[/ANG]

[EN]in that battle fell the power and dignity of the medes.[/EN]
[ANG]and on ðæm gefeohte meða cræft and heora duguð gefeoll.[/ANG]

[EN]46 shillings shall be given as compensation for a wound on the head, if both bones are pierced.[/EN]
[ANG]xl. heafodwunde to bote, gif ða ban beoð butu ðyrel, xxx scillinga geselle him mon.[/ANG]


[EN]In that moment, one of the twelve companions consulte